In [1]:
import os
from dotenv import dotenv_values
config = dotenv_values(".env")
data_path = os.path.normpath(config['DATA_PATH'])
os.environ['DATA_PATH'] = data_path
print(f"DATA_PATH is set to: {os.environ['DATA_PATH']}")

from helpers import *


DATA_PATH is set to: /mnt/big_gulp/nc4_rat_data/Maze_Rats


In [ ]:
rat_number = 18
date = '250310'

msg_list = get_log(rat_number, date)
msg_list.sort(key=lambda x: x[0])
msg_list


Using path: /mnt/big_gulp/nc4_rat_data/Maze_Rats
Using path: /mnt/big_gulp/nc4_rat_data/Maze_Rats
Looking in folder: /mnt/big_gulp/nc4_rat_data/Maze_Rats/NC40018
Looking in folder: /mnt/big_gulp/nc4_rat_data/Maze_Rats/NC40018/250310/Raw/ROS


[[1741661332155980026, 'Subscribing to /Esmacat_read_sync_ease'],
 [1741661332156064135, 'Subscribing to /Esmacat_read_gantry_ease'],
 [1741661332156074745, 'Subscribing to /Esmacat_read_maze_ease'],
 [1741661332156084025, 'Subscribing to /natnet_ros/Gantry/pose'],
 [1741661332156090085, 'Subscribing to /natnet_ros/Gantry/marker0/pose'],
 [1741661332156092475, 'Subscribing to /natnet_ros/Gantry/marker1/pose'],
 [1741661332156095185, 'Subscribing to /natnet_ros/Gantry/marker2/pose'],
 [1741661332156105125, 'Subscribing to /natnet_ros/Harness/pose'],
 [1741661332156107445, 'Subscribing to /natnet_ros/Harness/marker0/pose'],
 [1741661332156112315, 'Subscribing to /natnet_ros/Harness/marker1/pose'],
 [1741661332156115465, 'Subscribing to /natnet_ros/Harness/marker2/pose'],
 [1741661332156117305, 'Subscribing to /natnet_ros/MazeBoundary/pose'],
 [1741661332156119385, 'Subscribing to /natnet_ros/MazeBoundary/marker0/pose'],
 [1741661332156122315, 'Subscribing to /natnet_ros/MazeBoundary/mark

In [19]:
# Filter messages
trial_start_msg_list = [msg for msg in msg_list if msg[1].startswith('Current trial number:')]
trial_start_msg_list

trial_end_msg_list = [msg for msg in msg_list if msg[1].startswith('INTER_TRIAL_INTERVAL')]
trial_end_msg_list



[[1741659010747037609, 'INTER_TRIAL_INTERVAL'],
 [1741659069036904557, 'INTER_TRIAL_INTERVAL'],
 [1741659099717657155, 'INTER_TRIAL_INTERVAL'],
 [1741659297291432715, 'INTER_TRIAL_INTERVAL'],
 [1741659373703859042, 'INTER_TRIAL_INTERVAL'],
 [1741659432073978910, 'INTER_TRIAL_INTERVAL'],
 [1741659513784160187, 'INTER_TRIAL_INTERVAL'],
 [1741659547224713164, 'INTER_TRIAL_INTERVAL'],
 [1741659624674059358, 'INTER_TRIAL_INTERVAL'],
 [1741659670564337701, 'INTER_TRIAL_INTERVAL'],
 [1741659717273904850, 'INTER_TRIAL_INTERVAL'],
 [1741659781944051968, 'INTER_TRIAL_INTERVAL'],
 [1741659821254493702, 'INTER_TRIAL_INTERVAL'],
 [1741659873114470059, 'INTER_TRIAL_INTERVAL'],
 [1741659927994375861, 'INTER_TRIAL_INTERVAL'],
 [1741659964874826663, 'INTER_TRIAL_INTERVAL'],
 [1741660006815467116, 'INTER_TRIAL_INTERVAL'],
 [1741660048215282653, 'INTER_TRIAL_INTERVAL'],
 [1741660234874010095, 'INTER_TRIAL_INTERVAL'],
 [1741660272194673511, 'INTER_TRIAL_INTERVAL'],
 [1741660315584252192, 'INTER_TRIAL_INTE

In [20]:
# Construct trial list
trial_list = []
k=1
for msg in trial_end_msg_list:
    # Find the corresponding trial start message whose timestamp is smaller than the trial end message
    trial_start_msg = [msg_start for msg_start in trial_start_msg_list if msg_start[0] < msg[0]][-1]
    trial_list.append([k, trial_start_msg[0], msg[0]])
    k += 1

trial_list

# Get duration of each trial
for trial in trial_list:
    trial.append((trial[2] - trial[1])*1.0/1e9)

trial_list

[[1, 1741658921977109052, 1741659010747037609, 88.769928557],
 [2, 1741659011767274436, 1741659069036904557, 57.269630121],
 [3, 1741659070057380277, 1741659099717657155, 29.660276878],
 [4, 1741659100737323817, 1741659297291432715, 196.554108898],
 [5, 1741659298301973246, 1741659373703859042, 75.401885796],
 [6, 1741659374714045791, 1741659432073978910, 57.359933119],
 [7, 1741659433084077337, 1741659513784160187, 80.70008285],
 [8, 1741659514794734644, 1741659547224713164, 32.42997852],
 [9, 1741659548244279456, 1741659624674059358, 76.429779902],
 [10, 1741659625684626321, 1741659670564337701, 44.87971138],
 [11, 1741659671575174981, 1741659717273904850, 45.698729869],
 [12, 1741659718285033096, 1741659781944051968, 63.659018872],
 [13, 1741659782964346810, 1741659821254493702, 38.290146892],
 [14, 1741659822264860810, 1741659873114470059, 50.849609249],
 [15, 1741659874124979901, 1741659927994375861, 53.86939596],
 [16, 1741659929004112113, 1741659964874826663, 35.87071455],
 [17,

In [27]:
# Find messages within a trial that contains 'Selected stimulus is:'

#selected_stimulus_msg_list = [msg for msg in msg_list if msg[1].startswith('Selected stimulus is:')]

#for msg in selected_stimulus_msg_list:
    # Find the corresponding trial
    #trial = [trial for trial in trial_list if trial[1] < msg[0] and trial[2] > msg[0]][0]
    #print(f"Trial {trial[0]}: {msg[1]}")